ANGLE

In [30]:
import tensorflow as tf 
from keras import layers, models, optimizers, losses, ops
import numpy as np
import pandas as pd 
import re
import os

image_size = (244,244)
input_shape = (244,244,3)
batch_size = 64

class Angle_Predictor():
    
    def __init__(self):
            self.get_predictions()
    
    def get_predictions(self):
       
        model_angle = models.load_model('model_angle.model.keras')

        #create dictionary for tracking predictions
        predictions = {'image_id' : [], 'angle' : []}#, 'speed' : []}

        #categories for prediction, speed and angle 
        categories_angle = [0.0, 0.0625, 0.125, 0.1875, 0.25, 0.3125, 0.375, 0.4375, 0.5, 0.5625, 0.625, 0.6875, 0.75, 0.8125, 0.875, 0.9375, 1.0]
        #categories_speed = [0, 1]

        test_images_angle = tf.keras.preprocessing.image_dataset_from_directory('data/test_data_2/angle/', 
                                                                            labels = 'inferred', 
                                                                            label_mode = 'categorical',
                                                                            batch_size = batch_size,
                                                                            image_size = image_size,
                                                                            shuffle = False)


        paths = test_images_angle.file_paths

        angle_predictions = model_angle.predict(test_images_angle)
        #speed_predictions = model_speed.predict(test_images)

        #loop through test data and produce predictions 
        for index in range(len(angle_predictions)):
            
            
            angle_pred = categories_angle[np.argmax(angle_predictions[index])]
            
            # predictions['speed'].append('...')
            predictions['angle'].append(angle_pred)
            predictions['image_id'].append(re.findall(r'\d+', paths[index])[-1])
            
        #save predictions and as a csv 
        df = pd.DataFrame(predictions)
        df.to_csv('scott_angle_outputs.csv', index = False)

         
    

if __name__ == "__main__":
    Angle_Predictor()

Found 894 files belonging to 17 classes.
14/14 ━━━━━━━━━━━━━━━━━━━━ 47s 3s/step


In [31]:
import csv
from collections import defaultdict

# Initialize the dictionary with default list values
image_dict = defaultdict(list)

# Open the CSV file
with open('scott_angle_outputs.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    
    # Iterate through rows in the CSV file
    for row in reader:
        image_id = row['image_id']
        angle = float(row['angle'])  # Convert angle to float if needed
        image_dict[image_id].append(angle)

# Convert defaultdict to a regular dictionary (optional)
final_dict = dict(image_dict)



sorted_prediction_dict = dict(sorted(final_dict.items()))
print(sorted_prediction_dict)


#print(final_dict)


{'1': [0.625], '10': [0.3125], '100': [0.4375], '1001': [0.5], '1004': [0.625], '101': [0.5], '1010': [0.6875], '1012': [0.6875], '1013': [0.4375], '1016': [0.4375], '1019': [0.3125], '102': [0.625], '1023': [0.4375], '1024': [0.5], '1029': [0.4375], '103': [0.5], '1034': [0.625], '1036': [0.4375], '1037': [0.3125], '104': [0.4375], '105': [0.6875], '1051': [0.6875], '106': [0.625], '1061': [0.4375], '1063': [0.625], '1066': [0.6875], '107': [0.125], '1070': [0.625], '1073': [0.625], '1076': [0.6875], '108': [0.3125], '1081': [0.6875], '1083': [0.625], '1087': [0.5625], '109': [0.625], '1092': [0.6875], '1096': [0.625], '1097': [0.6875], '1099': [0.625], '110': [0.4375], '1100': [0.3125], '1106': [0.1875], '1107': [0.4375], '111': [0.4375], '1112': [0.625], '1115': [0.625], '112': [0.4375], '1123': [0.6875], '1124': [0.6875], '1125': [0.3125], '1128': [0.4375], '113': [0.3125], '1131': [0.4375], '114': [0.6875], '1140': [0.5], '1149': [0.3125], '115': [0.4375], '1154': [0.4375], '116':

In [32]:
import os
from collections import defaultdict

def folder_to_image_angle_dict(root_folder):
    image_angle_dict = defaultdict(list)
    
    for angle_folder in os.listdir(root_folder):
        angle_path = os.path.join(root_folder, angle_folder)
        if os.path.isdir(angle_path):
            try:
                angle = float(angle_folder)
                for image_file in os.listdir(angle_path):
                    # Skip hidden files within the angle folder
                    if image_file.startswith('.'):
                        continue

                    image_id = os.path.splitext(image_file)[0]
                    image_angle_dict[image_id].append(angle)
            except ValueError:
                # Skip folders that can't be converted to float
                continue
    
    return dict(image_angle_dict)

# Usage
root_folder = "data/test_data_2/angle"
result = folder_to_image_angle_dict(root_folder)
#print(result)

sorted_actual_dict = dict(sorted(result.items()))
print("sorted_actual_dict",sorted_actual_dict)
print("sorted_prediction_dict",sorted_prediction_dict)

sorted_actual_dict {'1': [0.4375], '10': [0.5], '100': [0.5], '1001': [0.375], '1004': [0.8125], '101': [0.6875], '1010': [0.8125], '1012': [0.625], '1013': [0.8125], '1016': [0.5625], '1019': [0.25], '102': [0.75], '1023': [0.625], '1024': [0.625], '1029': [0.375], '103': [0.5625], '1034': [0.625], '1036': [0.375], '1037': [0.8125], '104': [0.5], '105': [0.875], '1051': [0.8125], '106': [0.75], '1061': [0.625], '1063': [0.625], '1066': [0.625], '107': [0.6875], '1070': [0.625], '1073': [0.8125], '1076': [0.625], '108': [0.75], '1081': [0.8125], '1083': [0.625], '1087': [0.625], '109': [0.5625], '1092': [0.625], '1096': [0.8125], '1097': [0.625], '1099': [0.8125], '110': [0.5], '1100': [0.625], '1106': [0.1875], '1107': [0.375], '111': [0.5], '1112': [0.8125], '1115': [0.8125], '112': [0.5], '1123': [0.625], '1124': [0.8125], '1125': [0.625], '1128': [0.375], '113': [0.5], '1131': [0.375], '114': [0.6875], '1140': [0.375], '1149': [0.625], '115': [0.9375], '1154': [0.375], '116': [0.68

In [33]:
import numpy as np

def calculate_angle_mse(actual_dict, predicted_dict):
    """
    Calculates the Mean Squared Error (MSE) between two dictionaries with 
    image_id as keys and lists of angles as values.

    Args:
        actual_dict (dict): Dictionary containing actual angles for image_ids.
        predicted_dict (dict): Dictionary containing predicted angles for image_ids.

    Returns:
        float: The Mean Squared Error between the two dictionaries.
    """
    squared_errors = []

    # Iterate through the union of keys in both dictionaries
    for image_id in set(actual_dict.keys()) | set(predicted_dict.keys()):
        actual_angles = actual_dict.get(image_id, [])
        predicted_angles = predicted_dict.get(image_id, [])

        # Ensure both lists are not empty before calculating errors
        if actual_angles and predicted_angles:
            # Calculate squared errors for all pairs of angles
            errors = [(pred_angle - actual_angle) ** 2
                      for pred_angle in predicted_angles
                      for actual_angle in actual_angles]
            squared_errors.extend(errors)

    # Calculate MSE only if there are squared errors
    if squared_errors:
        angle_mse = np.mean(squared_errors)
    else:
        angle_mse = 0.0  # Or return np.nan, depending on how you want to handle empty data

    return angle_mse


# Example Usage (replace with your actual dictionaries)
# Assuming sorted_actual_dict and sorted_prediction_dict are already defined
actual_dict = sorted_actual_dict
predicted_dict = sorted_prediction_dict


angle_mse = calculate_angle_mse(actual_dict, predicted_dict)
print(f"Mean Squared Errorfor angle: {angle_mse:.6f}")


Mean Squared Errorfor angle: 0.039456


SPEED

In [34]:
import tensorflow as tf 
from keras import layers, models, optimizers, losses, ops
import numpy as np
import pandas as pd 
import re
import os

image_size = (244,244)
input_shape = (244,244,3)
batch_size = 64

class Speed_Predictor():
    
    def __init__(self):
            self.get_predictions()
    
        
    def get_predictions(self):
        # load speed model for prediction
        model_speed = models.load_model('model_speed.model.keras')

        #create dictionary for tracking predictions
        predictions = {'image_id' : [],'speed' : []}# ,'angle' : []}

        #categories for prediction, speed and angle 
        categories_speed = [0, 1]

        
        test_images_speed = tf.keras.preprocessing.image_dataset_from_directory('data/test_data_2/speed/', 
                                                                    labels = 'inferred', 
                                                                    label_mode = 'categorical',
                                                                    batch_size = batch_size,
                                                                    image_size = image_size,
                                                                    shuffle = False)

        paths = test_images_speed.file_paths

        #angle_predictions = model_angle.predict(test_images_angle)
        speed_predictions = model_speed.predict(test_images_speed)

        #loop through test data and produce predictions 
        for index in range(len(speed_predictions)):
            
            
            #angle_pred = categories_angle[np.argmax(angle_predictions[index])]
            speed_pred = categories_speed[np.argmax(speed_predictions[index])]
            
            predictions['speed'].append(speed_pred)
            #predictions['angle'].append('...')
            predictions['image_id'].append(re.findall(r'\d+', paths[index])[-1])
            
        #save predictions and as a csv 
        df = pd.DataFrame(predictions)
        df.to_csv('scott_speed_outputs.csv', index = False)

    

if __name__ == "__main__":
    Speed_Predictor()

Found 1040 files belonging to 2 classes.
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step


In [40]:
import csv
from collections import defaultdict

# Initialize the dictionary with default list values
speed_image_dict = defaultdict(list)

# Open the CSV file
with open('scott_speed_outputs.csv', 'r') as csvfile:
    reader = csv.DictReader(csvfile)
    
    # Iterate through rows in the CSV file
    for row in reader:
        image_id = row['image_id']
        speed = float(row['speed'])  # Convert speed to float if needed
        speed_image_dict[image_id].append(speed)


# Convert defaultdict to a regular dictionary (optional)
final_speed_dict = dict(speed_image_dict)



sorted_speed_prediction_dict = dict(sorted(final_speed_dict.items()))
print(sorted_speed_prediction_dict)



{'1': [0.0], '10': [1.0], '100': [1.0], '1000': [0.0], '1001': [1.0], '1002': [1.0], '1003': [1.0], '1004': [1.0], '1005': [1.0], '1006': [1.0], '1007': [0.0], '1008': [0.0], '1009': [1.0], '101': [1.0], '1010': [0.0], '1011': [1.0], '1012': [1.0], '1013': [1.0], '1014': [1.0], '1015': [1.0], '1016': [1.0], '1017': [1.0], '1018': [1.0], '1019': [0.0], '102': [1.0], '1020': [1.0], '1022': [1.0], '1023': [1.0], '1025': [1.0], '1026': [1.0], '1027': [1.0], '1028': [1.0], '1029': [1.0], '103': [0.0], '1031': [1.0], '1032': [1.0], '1033': [1.0], '1034': [1.0], '1035': [1.0], '1036': [1.0], '1037': [1.0], '104': [1.0], '1040': [1.0], '1041': [1.0], '1042': [1.0], '1044': [1.0], '1045': [1.0], '1046': [1.0], '1047': [1.0], '105': [1.0], '106': [1.0], '107': [1.0], '108': [1.0], '109': [0.0], '11': [0.0], '110': [1.0], '111': [1.0], '112': [1.0], '113': [1.0], '114': [1.0], '115': [0.0], '116': [0.0], '117': [1.0], '118': [1.0], '119': [1.0], '12': [1.0], '120': [0.0], '121': [0.0], '122': [1.

In [41]:
import os
from collections import defaultdict

def folder_to_image_speed_dict(root_folder):
    image_speed_dict = defaultdict(list)
    
    for speed_folder in os.listdir(root_folder):
        speed_path = os.path.join(root_folder, speed_folder)
        if os.path.isdir(speed_path):
            try:
                speed = float(speed_folder)
                for image_file in os.listdir(speed_path):
                    # Skip hidden files within the angle folder
                    if image_file.startswith('.'):
                        continue

                    image_id = os.path.splitext(image_file)[0]
                    image_speed_dict[image_id].append(speed)
            except ValueError:
                # Skip folders that can't be converted to float
                continue
    
    return dict(image_speed_dict)

# Usage
root_folder = "data/test_data_2/speed"
speed_result = folder_to_image_speed_dict(root_folder)
#print(result)

sorted_actual_speed_dict = dict(sorted(speed_result.items()))
print("sorted_actual_speed_dict",sorted_actual_speed_dict)
print("sorted_speed_prediction_dict",sorted_speed_prediction_dict)

sorted_actual_speed_dict {'1': [0.0], '10': [0.0], '100': [1.0], '1000': [0.0], '1001': [1.0], '1002': [1.0], '1003': [1.0], '1004': [1.0], '1005': [1.0], '1006': [1.0], '1007': [0.0], '1008': [0.0], '1009': [1.0], '101': [1.0], '1010': [0.0], '1011': [1.0], '1012': [1.0], '1013': [1.0], '1014': [1.0], '1015': [1.0], '1016': [1.0], '1017': [1.0], '1018': [1.0], '1019': [0.0], '102': [1.0], '1020': [1.0], '1022': [1.0], '1023': [1.0], '1025': [1.0], '1026': [1.0], '1027': [1.0], '1028': [1.0], '1029': [1.0], '103': [0.0], '1031': [1.0], '1032': [1.0], '1033': [1.0], '1034': [1.0], '1035': [1.0], '1036': [1.0], '1037': [1.0], '104': [1.0], '1040': [1.0], '1041': [1.0], '1042': [1.0], '1044': [1.0], '1045': [1.0], '1046': [1.0], '1047': [1.0], '105': [1.0], '106': [1.0], '107': [1.0], '108': [1.0], '109': [0.0], '11': [0.0], '110': [1.0], '111': [1.0], '112': [1.0], '113': [1.0], '114': [1.0], '115': [0.0], '116': [0.0], '117': [1.0], '118': [1.0], '119': [1.0], '12': [1.0], '120': [0.0],

In [ ]:
MSE

In [42]:
import numpy as np

def calculate_mse(actual_dict, predicted_dict):
    """
    Calculates the Mean Squared Error (MSE) between two dictionaries with 
    image_id as keys and lists of angles as values.

    Args:
        actual_dict (dict): Dictionary containing actual angles for image_ids.
        predicted_dict (dict): Dictionary containing predicted angles for image_ids.

    Returns:
        float: The Mean Squared Error between the two dictionaries.
    """
    squared_errors = []

    # Iterate through the union of keys in both dictionaries
    for image_id in set(actual_dict.keys()) | set(predicted_dict.keys()):
        actual_speeds = actual_dict.get(image_id, [])
        predicted_speeds = predicted_dict.get(image_id, [])

        # Ensure both lists are not empty before calculating errors
        if actual_speeds and predicted_speeds:
            # Calculate squared errors for all pairs of angles
            errors = [(pred_speed - actual_speed) ** 2
                      for pred_speed in predicted_speeds
                      for actual_speed in actual_speeds]
            squared_errors.extend(errors)


    # Calculate MSE only if there are squared errors
    if squared_errors:
        speed_mse = np.mean(squared_errors)
    else:
        speed_mse = 0.0  # Or return np.nan, depending on how you want to handle empty data

    return speed_mse


# Example Usage (replace with your actual dictionaries)
# Assuming sorted_actual_dict and sorted_prediction_dict are already defined
actual_dict = sorted_actual_speed_dict
predicted_dict = sorted_speed_prediction_dict




speed_mse = calculate_mse(actual_dict, predicted_dict)
print(f"Mean Squared Error for speed: {speed_mse:.6f}")


Mean Squared Error for speed: 0.018269


In [43]:
total_mse= (angle_mse+speed_mse)/2
print(f"Total Mean Squared Error: {total_mse:.6f}")

Total Mean Squared Error: 0.028862


In [161]:
if __name__ == "__main__":
    Angle_Predictor()
    Speed_Predictor() 
    speed_mse
    angle_mse  
    total_mse


print(f"Total Mean Squared Error: {total_mse:.6f}")

Found 894 files belonging to 17 classes.
14/14 ━━━━━━━━━━━━━━━━━━━━ 45s 3s/step
Found 1040 files belonging to 2 classes.
17/17 ━━━━━━━━━━━━━━━━━━━━ 54s 3s/step
Total Mean Squared Error: 0.121837


In [ ]:
# import numpy as np

# def compare_dictionaries(actual_dict, predicted_dict, n=10):
#     """
#     Compares two dictionaries with image_id as keys and lists of angles as values.
#     Prints the image_ids with the n greatest average absolute discrepancies,
#     along with the predicted and actual angles.

#     Args:
#         actual_dict (dict): Dictionary containing actual angles for image_ids.
#         predicted_dict (dict): Dictionary containing predicted angles for image_ids.
#         n (int): Number of image_ids with the greatest discrepancies to print.
#     """

#     discrepancies = {}

#     for image_id in set(actual_dict.keys()) | set(predicted_dict.keys()):  # Union of keys
#         actual_angles = actual_dict.get(image_id, [])
#         predicted_angles = predicted_dict.get(image_id, [])  # Get list of predicted angles

#         if actual_angles and predicted_angles:  # Check if BOTH lists are not empty
#             # Calculate the average absolute difference between ALL pairs
#             all_abs_diffs = []
#             for pred_angle in predicted_angles:
#                 for actual_angle in actual_angles:
#                     all_abs_diffs.append(abs(pred_angle - actual_angle))

#             avg_abs_diff = np.mean(all_abs_diffs) if all_abs_diffs else 0.0
#             discrepancies[image_id] = avg_abs_diff
#         else:
#             # Handle cases where either actual or predicted angles are missing.
#             if not actual_angles:
#                 print(f"Warning: Image ID {image_id} has no actual angles.")
#             if not predicted_angles:
#                 print(f"Warning: Image ID {image_id} has no predicted angles.")
#             discrepancies[image_id] = np.nan  # Or some other suitable missing value

#     # Sort discrepancies by average absolute difference in descending order, ignoring NaNs
#     sorted_discrepancies = sorted([(k, v) for k, v in discrepancies.items() if not np.isnan(v)],
#                                   key=lambda item: item[1], reverse=True)

#     # Print the top n discrepancies
#     print(f"Top {n} Image IDs with Greatest Average Absolute Discrepancies:")
#     for image_id, avg_abs_diff in sorted_discrepancies[:n]:
#         actual_angles_str = ", ".join(f"{angle:.6f}" for angle in actual_angles)
#         predicted_angles_str = ", ".join(f"{angle:.6f}" for angle in predicted_angles)
#         print(f"Image ID: {image_id}, Average Absolute Discrepancy: {avg_abs_diff:.4f}, "
#               f"Predicted Angles: [{predicted_dict[image_id]}], Actual Angles: [{actual_dict[image_id]}]")
        




# actual_dict = sorted_actual_dict
# predicted_dict = sorted_prediction_dict

# print("sorted_prediction_dict:", sorted_prediction_dict)  # Add this line
# actual_dict = sorted_actual_dict
# predicted_dict = sorted_prediction_dict
# compare_dictionaries(actual_dict, predicted_dict, n=50)

# print(actual_dict['313'])
# print(predicted_dict['313'])



sorted_prediction_dict: {'1': [0.4375], '10': [0.5], '100': [0.5], '1001': [0.375], '1004': [0.8125], '101': [0.625], '1010': [0.75], '1012': [0.625], '1013': [0.8125], '1016': [0.4375], '1019': [0.375], '102': [0.875], '1023': [0.625], '1024': [0.6875], '1029': [0.375], '103': [0.5625], '1034': [0.625], '1036': [0.4375], '1037': [0.4375], '104': [0.5], '105': [0.6875], '1051': [0.8125], '106': [0.8125], '1061': [0.5], '1063': [0.75], '1066': [0.625], '107': [0.6875], '1070': [0.625], '1073': [0.875], '1076': [0.6875], '108': [0.75], '1081': [0.8125], '1083': [0.625], '1087': [0.5625], '109': [0.625], '1092': [0.6875], '1096': [0.8125], '1097': [0.6875], '1099': [0.8125], '110': [0.5625], '1100': [0.625], '1106': [0.125], '1107': [0.5], '111': [0.5], '1112': [0.875], '1115': [0.8125], '112': [0.5], '1123': [0.75], '1124': [0.8125], '1125': [0.75], '1128': [0.4375], '113': [0.375], '1131': [0.375], '114': [0.625], '1140': [0.5], '1149': [0.8125], '115': [0.9375], '1154': [0.375], '116':